Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

In [1]:
%load_ext sql
%sql sqlite:///lab3-1.db

'Connected: @lab3-1.db'

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [5]:
%%sql
Pragma foreign_key=on;

DROP TABLE if exists Positions;
CREATE TABLE Positions
(
    id integer(0,0) NOT NULL Primary key,
    name varchar(0,0)
); 

DROP TABLE if exists Coaches;
CREATE TABLE Coaches
(
    id integer(0,0) NOT NULL Primary key,
    name varchar(0,0)
);

DROP TABLE if exists Teams;
CREATE TABLE Teams
(
    id integer(0,0) NOT NULL Primary key,
    name varchar(0,0),
    location varchar(0,0),
    coach_id intrger(0,0),
    FOREIGN KEY(coach_id) REFERENCES  Coaches(ID)
);

DROP TABLE if exists Players;
CREATE TABLE Players
(
    id integer(0,0) NOT NULL Primary key,
    name varchar(0,0),
    age integer(0,0),
    pos_id integer(0,0),
    team_id integer(0,0),
    start_date data,
    FOREIGN KEY(pos_id) REFERENCES  Positions(ID),
    FOREIGN KEY(team_id) REFERENCES  Teams(ID)
);

DROP TABLE if exists PlayersArchive ;
CREATE TABLE PlayersArchive 
(
    player_id integer(0,0),
    team_id integer(0,0),
    start_date data,
    end_date data,
    FOREIGN KEY(player_id) REFERENCES  Players(ID),
    FOREIGN KEY(team_id) REFERENCES  Teams(ID)
);

 * sqlite:///lab3-1.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [50]:
%%sql
INSERT INTO Positions(id, name) VALUES (1, 'вратарь');
INSERT INTO Positions(id, name) VALUES (2, 'защитник');
INSERT INTO Positions(id, name) VALUES (3, 'полузащитник');
INSERT INTO Positions(id, name) VALUES (4, 'нападающий');

 * sqlite:///lab3-1.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [51]:
import sqlite3
# создадим соединение с нашей базой данных
db = sqlite3.connect('lab3-1.db')

In [52]:
def insertCoach(name):
    try:
        # создаем курсор - это специальный объект который делает запросы и получает их результаты
        cur = db.cursor()
        cur.execute( '''SELECT COALESCE(MAX(id)+1, 1) FROM Coaches''') # увеличение id
        id = cur.fetchone()[0] # получение следующей строки данных или типNone, если объект не найден, возвразает кортеж
        cur.execute('''INSERT INTO Coaches(id, name) VALUES(?,?)''', (id, name))
        db.commit() # сохранение действий
    except TypeError:
        print('Error')
        return -1
    return id, name

In [53]:
insertCoach('Зидан')
insertCoach('Гвардиола')

(2, 'Гвардиола')

In [54]:
%%sql
Select *
From coaches

 * sqlite:///lab3-1.db
Done.


id,name
1,Зидан
2,Гвардиола


# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд


Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

In [55]:
def insertTeam(name, loc, coach_id):
    try:
        cur = db.cursor()
        cur.execute( '''SELECT COALESCE(MAX(id)+1, 1) FROM Teams''')
        id = cur.fetchone()[0]
        cur.execute('''INSERT INTO Teams(id, name, location, coach_id) VALUES(?,?,?,?)''', (id, name, loc, coach_id))
        db.commit()
    except TypeError:
        print('Error')
        return -1
    return id, name

In [56]:
insertTeam('Реал Мадрид', 'Мадрид', 1)
insertTeam('Манчестер сити', 'Манчестер', 2)

(2, 'Манчестер сити')

In [57]:
%%sql
SELECT *
FROM Teams

 * sqlite:///lab3-1.db
Done.


id,name,location,coach_id
1,Реал Мадрид,Мадрид,1
2,Манчестер сити,Манчестер,2


# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

In [58]:
def updateTeam(teamName, coachName = None, location = None):
    if (coachName != None and location != None):
        cur = db.cursor()
        cur.execute('''UPDATE Teams SET location = ?, 
                        coach_id = (SELECT id FROM Coaches WHERE name = ?)
                        WHERE name = ?''', (location, coachName, teamName))
        db.commit()
    elif (coachName != None and location == None):
        cur = db.cursor()
        cur.execute('''UPDATE Teams SET coach_id = (SELECT id FROM Coaches WHERE name = ?)
                         WHERE name = ?''', (coachName, teamName))
        db.commit()
    elif (coachName == None and location != None):
        cur = db.cursor()
        cur.execute('''UPDATE Teams SET location = ? 
                        WHERE name = ?''', (location, teamName))
        db.commit()
    elif (coachName == None and location == None):
        print('Error')
        return -1

In [59]:
insertCoach('Тухель')
insertTeam('ПСЖ', 'Париж', 3)
updateTeam('Реал Мадрид', None, 'Барселона')
updateTeam('Манчестер сити', 'Тухель', None)
updateTeam('ПСЖ', 'Гвардиола', 'Лион')

In [60]:
%%sql
SELECT *
FROM Teams

 * sqlite:///lab3-1.db
Done.


id,name,location,coach_id
1,Реал Мадрид,Барселона,1
2,Манчестер сити,Манчестер,3
3,ПСЖ,Лион,2


# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

In [61]:
import datetime
def insertPlayer(name, age, posName, teamName, startPlay = None):
    
    # проверим возраст игрока
    if (age < 10 or age > 50):
        print('Error: wrong age!')
        return -1
    
    cur = db.cursor()
    
    # проверим позицию игрока
    cur.execute('''SELECT id FROM Positions WHERE name = ?''', (posName,)) # выведет, если есть позиция
    try:
        posID = cur.fetchone()[0]
    except TypeError:
        print('Error: wrong position!')
        return -1

    # проверим название команды игрока
    cur.execute('''SELECT id FROM Teams WHERE name = ?''', (teamName,)) # выведет, если есть позиция
    try:
        teamID = cur.fetchone()[0]
    except TypeError:
        print('Error: wrong team name!')
        return -1
    
    # проверим название команды игрока
    if (startPlay == None):
        startPlay = date.today()
    else:
        try:
            datetime.datetime.strptime(startPlay,"%Y-%m-%d")
        except ValueError:
            print('Error: wrong date!')
            return -1
        
    cur.execute( '''SELECT COALESCE(MAX(id)+1, 1) FROM Players''')
    id = cur.fetchone()[0]
    cur.execute('''INSERT INTO Players(id, name, age, pos_id, team_id, start_date) VALUES(?,?,?,?,?,?)''',
    (id, name, age, posID, teamID, startPlay))
    db.commit()
    print("Successfully added!")

In [62]:
insertPlayer('Роналдо', 32, 'Нападающий', 'Реал Мадрид', None)
insertPlayer('Роналдо', 54, 'Нападающий', 'Реал Мадрид', None)
insertPlayer('Роналдо', 32, 'Нападающий', 'Мадрид', None)
insertPlayer('Роналдо', 32, 'нападающий', 'Реал Мадрид', '111-12')
insertPlayer('Роналдо', 32, 'нападающий', 'Реал Мадрид', '2009-12-12')

Error: wrong position!
Error: wrong age!
Error: wrong position!
Error: wrong date!
Successfully added!


In [63]:
%%sql
SELECT *
FROM Players

 * sqlite:///lab3-1.db
Done.


id,name,age,pos_id,team_id,start_date
1,Роналдо,32,4,1,2009-12-12


# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.

In [64]:
import datetime
def transaction_team_player(player,old_team,new_team,start_date_new_team):

    old_team_id = check_team(old_team)
    if old_team_id==-1:
        return -1

    new_team_id = check_team(new_team)
    if new_team_id==-1:
        return -1
    player_id = check_player(player,old_team_id)
    if player_id==-1:
        return -1

    cur = db.cursor()
    cur.execute('''select start_date from Players where ID = ?''', (player_id,))
    start_date_old_team=cur.fetchone()[0]

    end_date_old_team=datetime.datetime.strptime(start_date_new_team,"%Y-%m-%d")
    end_date_old_team=(end_date_old_team - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

    try:
        cur.execute('''insert into PlayersArchive(player_id, team_id, start_date, end_date) VALUES(?,?,?,?)''',
        (player_id,old_team_id,start_date_old_team,end_date_old_team))
        cur.execute('''Update Players set start_date = ?,team_id = ? where ID = ?''',
        (start_date_new_team,new_team_id,player_id))
        db.commit()
    except db.Error:
        print('FAILED!!!')
        db.rollback()

def check_player(player,team_id):
    cur = db.cursor()
    cur.execute('''select ID from Players where player_name = ? and team_id = ?''', (player,team_id))
    try:
        player_id = cur.fetchone()[0]
        return player_id
    except TypeError:
        print('This player does not exist')
        return -1

def check_team(old_team):
    cur = db.cursor()
    cur.execute('''select ID from Teams where team_name = ?''', (old_team,))
    try:
        team_id = cur.fetchone()[0]
        return team_id
    except TypeError:
        print('Such a team does not exist')
        return -1

In [65]:
import datetime
def playerTransfer(name, oldTeam ,newTeam, start_dateNewTeam):
    cur = db.cursor()

    # проверим существование игрока
    cur.execute('''SELECT id FROM Players WHERE name = ?''', (name,))
    try:
        playerID = cur.fetchone()[0]
    except TypeError:
        print(('Error: wrong player!'))
        return -1
    
    # проверим старую команду игрока
    cur.execute('''SELECT id FROM Teams WHERE name = ?''', (oldTeam,))
    try:
        oldTeamID = cur.fetchone()[0]
    except TypeError:
        print(('Error: wrong oldTeam!'))
        return -1
    
    # проверим новую команду игрока
    cur.execute('''SELECT id FROM Teams WHERE name = ?''', (newTeam,))
    try:
        newTeamID = cur.fetchone()[0]
    except TypeError:
        print(('Error: wrong newTeam!'))
        return -1
    
    cur.execute('''SELECT start_date FROM Players WHERE id = ?''', (playerID,))
    start_dateOldTeam=cur.fetchone()[0]

    end_dateOldTeam=datetime.datetime.strptime(start_dateNewTeam,"%Y-%m-%d")
    end_dateOldTeam=(end_dateOldTeam - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

    try:
        cur.execute('''INSERT INTO PlayersArchive(player_id, team_id, start_date, end_date) VALUES(?,?,?,?)''',
                    (playerID, oldTeamID, start_dateOldTeam, end_dateOldTeam))
        cur.execute('''UPDATE Players SET start_date = ?, team_id = ? WHERE id = ?''',
                    (start_dateNewTeam, newTeamID, playerID))
        db.commit()
    except db.Error:
        print('Error: transfer failed!')
        #db.rollback() # - отмена транзакции

In [66]:
#playerTransfer("Роналдо", "Реал Мадрид", "Манчестер сити", "2010-12-12")

In [67]:
%%sql
SELECT *
FROM PlayersArchive

 * sqlite:///lab3-1.db
Done.


player_id,team_id,start_date,end_date


In [68]:
%%sql
SELECT *
FROM Players

 * sqlite:///lab3-1.db
Done.


id,name,age,pos_id,team_id,start_date
1,Роналдо,32,4,1,2009-12-12


In [69]:
%%sql
SELECT *
FROM Teams

 * sqlite:///lab3-1.db
Done.


id,name,location,coach_id
1,Реал Мадрид,Барселона,1
2,Манчестер сити,Манчестер,3
3,ПСЖ,Лион,2


In [70]:
insertPlayer('1', 27, 'нападающий', 'Реал Мадрид', '2010-11-10')
playerTransfer("Роналдо", "Реал Мадрид", "Манчестер сити", "2011-12-12")
playerTransfer("Роналдо", "Манчестер сити", "ПСЖ", "2012-12-12")
playerTransfer("Роналдо", "ПСЖ", "Реал Мадрид", "2013-12-12")
playerTransfer("Роналдо", "Реал Мадрид", "Манчестер сити", "2014-12-12")
playerTransfer("Роналдо", "Манчестер сити", "ПСЖ", "2015-12-12")

Successfully added!


In [73]:
%%sql
SELECT *
FROM PlayersArchive

 * sqlite:///lab3-1.db
Done.


player_id,team_id,start_date,end_date
1,1,2009-12-12,2011-12-11
1,2,2011-12-12,2012-12-11
1,3,2012-12-12,2013-12-11
1,1,2013-12-12,2014-12-11
1,2,2014-12-12,2015-12-11


In [72]:
db.close()
